<a href="https://colab.research.google.com/github/Sayandip2023/SNUCV/blob/main/Copy_of_Sentiment_Analysis_using_Computer_Vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q kaggle
from google.colab import files

# Upload kaggle.json file
files.upload()

# Make a directory for Kaggle and move the kaggle.json file there
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/

# Set the permissions for kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [ ]:
# Download the dataset from Kaggle
!kaggle datasets download -d nishantathawale/fer-dataset-for-3-class-sentiment-analysis
# Unzip the downloaded dataset
!unzip fer-dataset-for-3-class-sentiment-analysis.zip

Streaming output truncated to the last 5000 lines.
  inflating: FER_Custom_Dataset/neutral/Training_73579501.jpg  
  inflating: FER_Custom_Dataset/neutral/Training_73593223.jpg  
  inflating: FER_Custom_Dataset/neutral/Training_73599260.jpg  
  inflating: FER_Custom_Dataset/neutral/Training_7360010.jpg  
  inflating: FER_Custom_Dataset/neutral/Training_7360259.jpg  
  inflating: FER_Custom_Dataset/neutral/Training_73606174.jpg  
  inflating: FER_Custom_Dataset/neutral/Training_73609242.jpg  
  inflating: FER_Custom_Dataset/neutral/Training_73619612.jpg  
  inflating: FER_Custom_Dataset/neutral/Training_73640022.jpg  
  inflating: FER_Custom_Dataset/neutral/Training_73650202.jpg  
  inflating: FER_Custom_Dataset/neutral/Training_73668673.jpg  
  inflating: FER_Custom_Dataset/neutral/Training_73673187.jpg  
  inflating: FER_Custom_Dataset/neutral/Training_73677468.jpg  
  inflating: FER_Custom_Dataset/neutral/Training_7368170.jpg  
  inflating: FER_Custom_Dataset/neutral/Training_7368650

In [ ]:
!pip install tensorflow_addons
import os
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, Dropout, LayerNormalization
from keras.callbacks import ReduceLROnPlateau
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
# Define the base directory where the dataset is located
base_dir = '/content/FER_Custom_Dataset'

# Define the class subfolders
classes = ['Disappointed', 'interested', 'neutral']

# Function to get file paths and labels
def get_file_paths_and_labels(base_dir, classes):
    file_paths = []
    labels = []
    for class_label, class_name in enumerate(classes):
        class_dir = os.path.join(base_dir, class_name)
        for root, _, files in os.walk(class_dir):
            for file in files:
                file_paths.append(os.path.join(root, file))
                labels.append(class_label)
    return np.array(file_paths), np.array(labels)

# Get file paths and labels
file_paths, labels = get_file_paths_and_labels(base_dir, classes)

# Split the data into training, validation, and test sets (80%, 10%, 10%)
train_paths, test_paths, train_labels, test_labels = train_test_split(
    file_paths, labels, test_size=0.2, stratify=labels, random_state=42)

val_paths, test_paths, val_labels, test_labels = train_test_split(
    test_paths, test_labels, test_size=0.5, stratify=test_labels, random_state=42)

# Check the number of samples in each set
print(f'Train samples: {len(train_paths)}')
print(f'Validation samples: {len(val_paths)}')
print(f'Test samples: {len(test_paths)}')

In [ ]:
# Plot data distribution
import pandas as pd

# Create a DataFrame for plotting
df = pd.DataFrame({
    'Set': ['Train', 'Validation', 'Test'],
    'Count': [len(train_paths), len(val_paths), len(test_paths)]
})

# Plot pie chart
plt.figure(figsize=(8, 8))
plt.pie(df['Count'], labels=df['Set'], autopct='%1.1f%%', startangle=140)
plt.title('Data Distribution')
plt.show()

# Function to convert image path to image array
def path_to_image_array(image_path):
    return img_to_array(load_img(image_path, target_size=(64, 64))) / 255.0

# Function to generate batches of images and labels
def data_generator(file_paths, labels, batch_size):
    while True:
        indices = np.arange(len(file_paths))
        np.random.shuffle(indices)
        for start in range(0, len(file_paths), batch_size):
            end = min(start + batch_size, len(file_paths))
            batch_indices = indices[start:end]
            batch_images = [path_to_image_array(file_paths[i]) for i in batch_indices]
            batch_labels = [labels[i] for i in batch_indices]
            yield np.array(batch_images), np.array(batch_labels)

# Parameters
batch_size = 32

# Create generators
train_generator = data_generator(train_paths, train_labels, batch_size)
val_generator = data_generator(val_paths, val_labels, batch_size)
test_generator = data_generator(test_paths, test_labels, batch_size)

# Calculate the number of steps per epoch for training, validation, and testing
train_steps = len(train_paths) // batch_size
val_steps = len(val_paths) // batch_size
test_steps = len(test_paths) // batch_size

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, Dropout, BatchNormalization

# Build the CNN model with Batch Normalization
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    Conv2D(256, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2, 2),

    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')
])

# Compile the model with AdamW optimizer
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=tf.keras.optimizers.AdamW(learning_rate=1e-3),
              metrics=['accuracy'])

# Callbacks
callbacks = [ReduceLROnPlateau(monitor='val_loss', factor=0.4, patience=3)]

# Train the CNN model
history = model.fit(
    train_generator,
    steps_per_epoch=train_steps,
    epochs=25,
    validation_data=val_generator,
    validation_steps=val_steps,
    callbacks=callbacks
)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_generator, steps=test_steps)
print('Test accuracy:', test_acc)

In [ ]:
# Plot training & validation accuracy/loss values
def plot_training_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(len(acc))

    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(epochs, acc, 'r', label='Training accuracy')
    plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
    plt.title('Training and validation accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs, loss, 'r', label='Training loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

    plt.show()

plot_training_history(history)

In [ ]:
# Save the model
model.save('cnn_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
